In [0]:
import sklearn, torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [0]:
X_target, y_target = sklearn.datasets.make_blobs(n_samples = 8000, n_features = 6, centers = 6 , random_state = 0, cluster_std = 1.5)
                                                #  centers = [[-5, 4], [-3, -2.5], [2, -10.5], [5, -13.5], [-2, 10.5], [0, 4], [2, -2], [4, -3]])

In [0]:
n_features = 6

In [0]:
# plt.scatter(X_target[:,0], X_target[:,1])

In [0]:
# Change the labels to binary labels
for ind, ele in enumerate(y_target):
  if ele < 3:
    y_target[ind] = 0
  else:
    y_target[ind] = 1

In [0]:
# Define the dataset class for a custom dataset
class CustomDataset(Dataset):
    
    def __init__(self, data_frame, transform=None):
        self.data = data_frame
        self.transform = transform
#         
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        obs = self.data.iloc[index, 0 : n_features].values
        label = self.data.iloc[index, n_features]
        
        if self.transform is not None:
            obs = self.transform(obs)
            
        return obs, label

In [0]:
# Transforming the data into Pytorch format
data_target = pd.DataFrame({
                   'X1' : X_target[:, 0].tolist(),
                   'X2' : X_target[:, 1].tolist(), 
                   'X3' : X_target[:, 2].tolist(),
                   'X4' : X_target[:, 3].tolist(),
                   'X5' : X_target[:, 4].tolist(),
                   'X6' : X_target[:, 5].tolist(),
                  #  'X7' : X_target[:, 6].tolist(),
                  #  'X8' : X_target[:, 7].tolist(),
                  #  'X9' : X_target[:, 8].tolist(),
                  #  'X10' : X_target[:, 9].tolist(),
                   'label' : y_target.tolist()})
# Plot the toy data
# plt.figure(figsize = (5, 5))
# plt.scatter(data_target['X1'], data_target['X2'])
# plt.xlabel('feature 1')
# plt.ylabel('feature 2')
# plt.xlim(-15, 15)
# plt.ylim(-15, 15)

In [0]:
target_dataset = CustomDataset(data_target, transform = None)
target_loader = torch.utils.data.DataLoader(target_dataset, batch_size = 64, shuffle = True, num_workers = 2)

In [0]:
#  import torch.nn as nn
#  import torch.nn.functional as F

#  class linear_model(nn.Module):
#    def __init__(self):
#      super(linear_model, self).__init__()
#      self.fc11 = nn.Linear(n_features, 1)
#      nn.init.xavier_normal_(self.fc11.weight)

#    def forward(self, x_):
#      x_ = x_.view(-1, n_features)
#      x1_ = torch.sigmoid(self.fc11(x_))  
    
#      return x1_
  
#  model = linear_model()  

In [0]:
# # # Define loss function and optimizer
# import torch.optim as optim

# loss_function = nn.BCELoss(reduction = 'mean') # sums all outputs and divides by total data points

# optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [0]:
# n_epochs = 5

# for epochs in range(n_epochs):
 
#   running_loss = 0
  
#   for batch in target_loader:
#     data, targets = batch

#     optimizer.zero_grad()

#     output = model(data.float()) 
#     loss = loss_function(output, targets.view(-1, 1).float())
  
#     loss.backward()

#     optimizer.step()

#     running_loss += loss.item()
#   print(running_loss)

In [0]:
# # Test performance on the entire test set
# correct = 0
# total = 0
# with torch.no_grad():
#   for data in target_loader:
#     images, labels = data
#     outputs = model(images.float())
# #     _, predicted = torch.max(outputs, 1)
#     predicted = outputs > 0.5
# #     predicted = predicted.view(predicted.size(0))
#     total += labels.size(0)
#     matches = 0
#     for i in range(len(predicted)):
#       if predicted[i].item() == labels[i].item():
#         matches += 1
#     correct += matches#(predicted == labels).sum().item()
#   print('Accuracy of the network on the entire data set is : %d %%' %(100 * correct/ total))

In [0]:
# # Save the idea weights (ideal for the target data)
# w_star = model.fc11.weight
# w_star

In [0]:
w_star = torch.Tensor([[-0.1564, -0.1547,  0.2450,  0.9513, -0.5191, -0.2096]])

In [0]:
# x_s1 = list(range(-20, 25, 5))
# y_s1 = []
# y_s2 = []
# for el in x_s1:
#   y_s1.append((-model.fc11.weight[0][0].item()/ model.fc11.weight[0][1].item()) * el)


# plt.plot(x_s1, y_s1, color = 'red') 

# plt.scatter(data_target['X1'], data_target['X2'])
# plt.xlabel('feature 1')
# plt.ylabel('feature 2')
# plt.xlim(-15, 15)
# plt.ylim(-15, 15)

In [0]:
def gram_schmidt(A):
    """Orthogonalize a set of vectors stored as the columns of matrix A."""
    # Get the number of vectors.
    n = A.shape[1]
    for j in range(n):
        # To orthogonalize the vector in column j with respect to the
        # previous vectors, subtract from it its projection onto
        # each of the previous vectors.
        for k in range(j):
            A[:, j] -= np.dot(A[:, k], A[:, j]) * A[:, k]
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
    return A


In [0]:
w_star1 =  np.reshape(np.array(w_star.detach()), (-1, 1))

# Normalized w*
w_star1 = w_star1 / np.linalg.norm(w_star1)

# denerating rest n_features - 1 number of vectors
rest = np.random.rand(n_features, n_features - 1)
A_ele = np.hstack([w_star1, rest])

# Obtain orthonormal matrix A
A_all_ele = gram_schmidt(A_ele)

In [0]:
eval_rest = [-0.24085888133933997,
1.3234005271655356,
0.20926702332798985,
-0.8625208931380124,
-1.4072147504944934]

A = np.zeros((n_features, n_features))

for i in range(n_features):
  if i == 0:
    A += 1 * np.dot(np.reshape(A_all_ele[:, i], (-1, 1)), np.reshape(A_all_ele[:, i], (1, -1)))
  else:
    # r = np.random.uniform(-2, 2)
    # print(r) 
    A += eval_rest[i - 1] * np.dot(np.reshape(A_all_ele[:, i], (-1, 1)), np.reshape(A_all_ele[:, i], (1, -1)))

In [0]:
# Create the source data
X_source = np.zeros(np.shape(X_target))

for ind, d in enumerate(X_target):
  if y_target[ind] == 0:
    X_source[ind] = np.reshape(np.dot(A, np.reshape(X_target[ind], (-1, 1))), (n_features, ))
  else:
    X_source[ind] = X_target[ind]

In [0]:
# ######### ALTERNATIVE WAY TO GENERATE SOURCE DATA
# X_source, y_source = sklearn.datasets.make_blobs(n_samples=2000, n_features=2, centers = [[-1,5], [1, -5]], cluster_std = 1)#[[-5, -5], [-10, 5], [10, -15], [5, 5], [15, 10], [2, 15]], cluster_std = 2.5)

In [0]:
# # A plot to compare targe and source data
# plt.scatter(X_target[:, 0], X_target[:, 1], label = 'target data', alpha = 0.5)
# plt.scatter(X_source[:, 0], X_source[:, 1], label = 'source data', alpha = 0.5)
# plt.legend()

In [0]:
# Transforming the data into Pytorch format
data_source = pd.DataFrame({
                   'X1' : X_source[:, 0].tolist(),
                   'X2' : X_source[:, 1].tolist(), 
                   'X3' : X_source[:, 2].tolist(),
                   'X4' : X_source[:, 3].tolist(), 
                   'X5' : X_source[:, 4].tolist(),
                   'X6' : X_source[:, 5].tolist(), 
                  #  'X7' : X_source[:, 6].tolist(),
                  #  'X8' : X_source[:, 7].tolist(), 
                  #  'X9' : X_source[:, 8].tolist(),
                  #  'X10' : X_source[:, 9].tolist(),
                   'label' : y_target.tolist()})

In [0]:
source_dataset = CustomDataset(data_source, transform = None)
source_loader = torch.utils.data.DataLoader(source_dataset, batch_size = 64, shuffle = True, num_workers = 2)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class linear_model_new(nn.Module):
  def __init__(self):
    super(linear_model_new, self).__init__()
    self.fc1 = nn.Linear(n_features, 1)
    nn.init.xavier_normal_(self.fc1.weight)

  def forward(self, x):
    x = x.view(-1, n_features)
    x1 = torch.sigmoid(self.fc1(x))  
    
    return x1
  
model_new = linear_model_new()  

In [0]:
# Define loss function and optimizer
import torch.optim as optim

loss_function_new = nn.BCELoss(reduction = 'mean') # sums all outputs and divides by total data points

optimizer_new = optim.Adam(model_new.parameters(), lr = 0.01)

In [0]:
# + mu_ * torch.matmul(a.view(1, n_features), model_new.fc1.weight.view(n_features, 1)) / torch.sqrt(torch.matmul(model_new.fc1.weight.view(1, -1), model_new.fc1.weight.view(-1, 1)))  

In [36]:
# Define hyper-parameters before training
n_epochs = 4# a = torch.randn(1, n_features)
# a = a / a.norm() 
a = torch.Tensor([0, 0, 0, 0, 0, 1])
# print(a)
mu_ = 1
for epochs in range(n_epochs):
 
  running_loss = 0
  
  for batch in source_loader:
    data, targets = batch

    optimizer_new.zero_grad()

    output = model_new(data.float()) 
    loss = loss_function_new(output, targets.view(-1, 1).float()) + mu_ * torch.matmul(a.view(1, n_features), model_new.fc1.weight.view(n_features, 1)) / torch.sqrt(torch.matmul(model_new.fc1.weight.view(1, -1), model_new.fc1.weight.view(-1, 1)))
    loss.backward()

    optimizer_new.step()

    running_loss += loss.item()
  print(running_loss)

-96.50715732574463
-96.50743854045868
-96.57351171970367
-96.62896990776062


In [38]:
# Test performance on the entire test set
correct = 0
total = 0
with torch.no_grad():
  for data in target_loader:
    images, labels = data
    outputs = model_new(images.float())
#     _, predicted = torch.max(outputs, 1)
    predicted = outputs > 0.5
#     predicted = predicted.view(predicted.size(0))
    total += labels.size(0)
    matches = 0
    for i in range(len(predicted)):
      if predicted[i].item() == labels[i].item():
        matches += 1
    correct += matches#(predicted == labels).sum().item()
  print('Accuracy of the network on the entire data set is : %d %%' %(100 * correct/ total))

Accuracy of the network on the entire data set is : 68 %


In [29]:
model_new.fc1.weight

Parameter containing:
tensor([[ 0.0679,  0.7840,  0.2331,  0.4097, -0.3047, -1.4530]],
       requires_grad=True)

In [0]:
# plt.figure(figsize = (7, 5))
# x_s1 = list(range(-20, 25, 5))
# y_s1 = []
# y_s2 = []
# for el in x_s1:
#   y_s1.append((-model_new.fc1.weight[0][0].item()/ model_new.fc1.weight[0][1].item()) * el)
# for el in x_s1:
#   y_s2.append((-model.fc11.weight[0][0].item()/ model.fc11.weight[0][1].item()) * el)

# plt.plot(x_s1, y_s2, label = 'trained on target') 
# plt.plot(x_s1, y_s1, label = 'trained on source') 

# plt.legend()
# plt.scatter(data_target['X1'], data_target['X2'], label = 'target data')
# plt.scatter(data_source['X1'], data_source['X2'], label = 'source data')
# plt.legend()
# plt.xlabel('feature 1')
# plt.ylabel('feature 2')

In [0]:
# a = e(2) # ~ 9 epochs
#  mu = 1
# source_acc = [0.90, 0.94, 0.93, 0.96, 0.94]
# target_acc = [0.88, 0.91, 0.91, 0.85. 0.82]
# w_trainedOnsource = [ 0.0410, -0.6910, -0.0764,  1.1474, -0.7760,  0.6488]

# a = e(3)
# mu = 0.001
# source_acc = [0.99, 0.99, 0.99, 0.99]
# target_acc = [0.66, 0.57, 0.62, 0.62]
# w_trainedOnsource = [-0.0873,  0.6620, -0.0401,  0.5173, -1.1122,  0.1431] 

# mu = 0.01 (6 epochs)
# source_acc = [0.99, 0.99, 0.99, 0.99]  
# target_acc =  [0.67, 0.84, 0.60, 0.66]
# w_trainedOnsource = [ 0.1480,  0.4519,  0.1193,  1.0103, -0.9674, -0.4127]

# mu = 0.1 (6 epochs)
# source_acc = [0.99, 0.99, 0.99, 0.99, 0.99]  
# target_acc =  [0.78, 0.67, 0.69, 0.70, 0.85]
# w_trainedOnsource = [-0.0824,  1.2047,  0.0071,  1.0271, -0.4988, -0.2717]

# mu = 1 (6 epochs)
# source_acc = [0.97, 0.96, 0.97, 0.96, 0.94]  
# target_acc =  [0.68, 0.70, 0.65, 0.69, 0.70]
# w_trainedOnsource = [ 0.6137,  0.7717, -1.1279,  0.9522, -0.4991,  0.7584]

# a = e(4)
# mu = 0.001
# source_acc = [0.99, 0.98, 1, 0.99, 0.99, 0.99]  
# target_acc = [0.88, 0.90, 0.72, 0.68, 0.69, 0.74]  
# w_trainedOnsource = [-0.5362, -0.6480,  0.1678,  1.3624, -0.5187,  1.2504]

# mu = 0.01
# source_acc = [0.98, 0.99, 0.99, 0.99, 0.99]  
# target_acc = [0.67, 0.68, 0.63, 0.77, 0.75]  
# w_trainedOnsource = [ 0.0973,  0.6850, -0.0120,  1.0858, -0.4377,  0.2935]

# mu = 0.1
# source_acc = [0.99, 0.99, 0.98, 0.99, 0.99]  
# target_acc = [0.50, 0.5, 0.56, 0.53, 0.50]  
# w_trainedOnsource = [-0.8707,  1.3800, -0.0254, -0.3425, -0.1094,  0.6709]

# mu = 1
# source_acc = [0.94, 0.93, 0.93, 0.94, 0.94]  
# target_acc = [0.55, 0.50, 0.5, 0.46, 0.47]  
# w_trainedOnsource = [-0.9348,  0.6168,  0.1058, -0.7402, -0.1509,  0.7958]

# a = e(5)
# mu = 0.001
# source_acc = [0.99, 0.99, 0.99, 0.99, 0.99]  
# target_acc = [0.67, 0.66, 0.59, 0.55, 0.56]  
# w_trainedOnsource = [-0.0764,  0.5917,  0.2170,  0.5306, -0.4606, -0.6649] 

# mu = 0.01
# source_acc = [0.99, 0.99, 0.99, 0.99, 0.99]
# target_acc =  [0.67, 0.79, 0.84, 0.68, 0.66]
# w_trainedOnsource = [-0.6197,  0.8216,  0.2834,  0.9724, -0.5479,  0.3744]

# mu = 0.1
# source_acc = [0.99, 0.98, 0.96, 0.99, 0.9]
# target_acc =  [0.71, 0.82, 0.93, 0.73, 0.75]
# w_trainedOnsource = [-0.0790, -0.0834,  0.4029,  0.9120, -0.9244, -0.4637]

# mu = 1
# source_acc = [0.95, 0.95, 0.98, 0.94, 0.97]
# target_acc =  [0.74, 0.88, 0.66, 0.91, 0.67]
# w_trainedOnsource = [ 0.0434,  0.0444,  0.1794,  0.9025, -1.1929, -0.4635]

# a = e(6)
# mu = 0.001
# source_acc = [0.99, 0.99, 0.99, 0.99, 0.99]  
# target_acc = [0.56, 0.55, 0.52, 0.68, 0.55]  
# w_trainedOnsource = [-0.1897,  1.2013,  0.2390,  0.6079, -0.7662, -0.7045]

# mu = 0.01
# source_acc = [0.98, 0.99, 0.99, 0.99, 0.99]
# target_acc =  [0.91, 0.62, 0.88, 0.87, 0.66]
# w_trainedOnsource = [ 0.0191,  0.1738,  0.1169,  0.7730, -0.4382,  0.0603] 

# mu = 0.1
# source_acc = [0.99, 0.99, 0.99, 0.99]
# target_acc =  [0.83,0.71, 0.85, 0.74]
# w_trainedOnsource = [-0.7426,  1.1060,  0.3794,  1.7157, -0.3459, -0.4759]

# mu = 1
# source_acc =  [0.98, 0.98, 0.98, 0.98, 0.97]
# target_acc =  [0.74, 0.74, 0.74, 0.69, 0.68]
# w_trainedOnsource = [ 0.0648,  0.5646,  0.4808,  0.6665, -0.6328, -1.5918] 